In [1]:
import os
import h5py
import ngmix
import galsim
import random
import numpy as np
import pandas as pd
from astropy.io import fits

In [2]:
def make_obs(im, psf_im, pix_scale, d_x, d_y):
    
    # make observation to prepare data for estimation
    
    psf_noise = 1.0e-10
    
    cen = (np.array(im.shape)-1.0)/2.0
    psf_cen = (np.array(psf_im.shape)-1.0)/2.0
    
    jacobian = ngmix.DiagonalJacobian(row = cen[0] + d_y/pix_scale, col = cen[1] + d_x/pix_scale, scale = pix_scale)
    psf_jacobian = ngmix.DiagonalJacobian(row = psf_cen[0], col = psf_cen[1], scale = pixel_scale)
    
    wt = im*0 + 1.0/noise_sigma**2
    psf_wt = psf_im*0 + 1.0/psf_noise**2
    
    psf_obs = ngmix.Observation(
        psf_im,
        weight = psf_wt,
        jacobian = psf_jacobian
    )
    
    obs = ngmix.Observation(
        im,
        weight = wt,
        jacobian=jacobian,
        psf=psf_obs
    )

    return obs

In [3]:
def perform_metacal(observation, rng):
    
    # run the deconvolving, shearing, reconvoling, noise correction and measuring process in metacalibration
    # return res and obs dictionary which contains measurement result after the above process
    
    weight_fwhm = 1.2
    fitter = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)
    psf_fitter = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)
    
    # run the measurement code on observations
    psf_runner = ngmix.runners.PSFRunner(fitter = psf_fitter)
    runner = ngmix.runners.Runner(fitter=fitter)
    
    boot = ngmix.metacal.MetacalBootstrapper(
        runner=runner, psf_runner=psf_runner,
        rng = rng_2,
        psf = 'gauss',
        types = ['noshear', '1p', '1m', '2p', '2m']
    )
    
    result, obs = boot.go(observation)
    
    return result, obs

In [4]:
def organize_result(res, obs, shear_type):
    
    #organize results obtained from perform_metacal into data array
    
    # specify name and their corresponding datatype
    datatype = [
        ('flags', 'i4'),
        ('shear_type', 'U7'),
        ('s2n', 'f8'),
        ('g', 'f8', 2),
        ('T', 'f8'),
        ('Tpsf', 'f8'),
    ]
    
    data = np.zeros(1, dtype = datatype)
    data['shear_type'] = shear_type
    data['flags'] = res['flags']
    
    # successful measurement or not
    if res['flags']==0: 
        data['s2n'] = res['s2n']
        data['g'] = res['e'] # ellipticity
        data['T'] = res['T'] #second moment
    else:
        data['s2n'] = np.nan
        data['g'] = np.nan
        data['T'] = np.nan
        data['Tpsf'] = np.nan
        
    data['Tpsf'] = obs.psf.meta['result']['T']
    
    return data

In [5]:
def data_index_selection(data, shear_type):
    
    # select indices in data array to extract wanted data
    
    w, = np.where((data['flags']==0) & (data['shear_type'] == shear_type))
    return w

In [6]:
def get_calibrated_result(data):
    
    # get calibrated shear
    
    w = data_index_selection(data=data, shear_type='noshear')
    w_1p = data_index_selection(data=data, shear_type='1p')
    w_1m = data_index_selection(data=data, shear_type='1m')
    w_2p = data_index_selection(data=data, shear_type='2p')
    w_2m = data_index_selection(data=data, shear_type='2m')
    
    """
    g_list = []
    for i in range(len(data['g'][w])):
        g1 = data['g'][w][i][0]   
        g1_1p = data['g'][w_1p, 0][i]
        g1_1m = data['g'][w_1m, 0][i]
        R11 = (g1_1p - g1_1m)/0.02
        g1 = g1/R11
        g_list.append(g1)

    
    #plt.hist(g_list, bins=20, color='skyblue', edgecolor='black')
    print (np.mean(g_list))
    print (data['s2n'][w].mean())
    """
    
    g = data['g'][w].mean(axis=0)
    gerr = data['g'][w].std(axis=0) / np.sqrt(w.size)
    g1_1p = data['g'][w_1p, 0].mean()
    g1_1m = data['g'][w_1m, 0].mean()
    R11 = (g1_1p - g1_1m)/0.02
    
    g1_2p = data['g'][w_1p, 0].mean()
    g1_2m = data['g'][w_1m, 0].mean()
    R22 = (g1_1p - g1_1m)/0.02
    
    shear = (g[0]/R11, g[1]/R22)
    shear_err = (gerr/R11, g[1]/R22)
    
    return shear, shear_err, 

In [9]:
hf = h5py.File('/data3/shear_simulated_galaxy/NonUniformPsf_image127x127_with_metadata.hdf5', 'r')

# Initialize an empty dictionary to store data
data_dict = {}

# Iterate through items in the HDF5 file
for name, dataset in hf.items():
    if isinstance(dataset, h5py.Dataset):
        # Preprocess the data as needed and add it to the dictionary
        if name == 'galaxy_image' or name == 'psf_image':
            # Skip image datasets for now
            continue
        else:
            data_dict[name] = dataset[:]
            
galaxy_images = hf['galaxy_image'][:]
psf_images = hf['psf_image'][:]
            
# Close the HDF5 file
hf.close()

# Convert the dictionary to a Pandas DataFrame
metadata_df = pd.DataFrame(data_dict)
"""
# Print the metadata DataFrame
print(metadata_df)

# Add 'galaxy_image' and 'psf_image' columns to the metadata DataFrame
metadata_df['galaxy_image'] = galaxy_images
metadata_df['psf_image'] = psf_images
"""
galaxy_image_list = []
for image in galaxy_images:
    galaxy_image_list.append(image)

psf_image_list = []
for pimage in psf_images:
    psf_image_list.append(pimage)
    
metadata_df['galaxy_image'] = galaxy_image_list
metadata_df['psf_image'] = psf_image_list

print (type(galaxy_image_list[0]))

# Now metadata_df contains both metadata and images matched by 'object_id'
# print(metadata_df)

<class 'numpy.ndarray'>


In [15]:
unique_galaxy_ids = metadata_df['galaxy_id'].unique()
pixel_scale = .2
noise_sigma = .001
seed_2 = 6424512
rng_2 = np.random.RandomState(seed_2)

for gid in unique_galaxy_ids:
    
    this_gal = []
    
    for index, row in metadata_df.iterrows():
        if gid == row['galaxy_id']:
            this_realization = {
                'galaxy_image': row['galaxy_image'],
                'psf_image': row['psf_image'],
                'dx': row['shift_radius_dx'],
                'dy': row['shift_radius_dy'],
                'true_shear': (row['g1'], row['g2'])
                }
            this_gal.append(this_realization)
    
    this_galaxy = pd.DataFrame(this_gal)
    data_list=[]
    
    for ind, rows in this_galaxy.iterrows():
        galaxy_image = rows['galaxy_image']
        psf_image = rows['psf_image']
        true_shear = rows['true_shear']
        d_x = rows['dx']
        d_y = rows['dy']

        obs = make_obs(galaxy_image, psf_image, pixel_scale, d_x, d_y)
        resdict, obsdict = perform_metacal(obs, rng_2)
    
        for ShearType, ShearRes in resdict.items():
            dt = organize_result(res=ShearRes, obs=obsdict[ShearType],shear_type=ShearType)
            data_list.append(dt)
                
    data = np.hstack(data_list)
    measured_shear, measured_shear_err = get_calibrated_result(data)
    
    print ("true shear")
    print (true_shear)
    print ("measured shear")
    print (measured_shear)

true shear
(0.0749715571291744, 0.0054833702743053)
measured shear
(-0.038229130207944786, 0.02313671774891978)
true shear
(-0.0445619521196931, -0.0539411823730915)
measured shear
(-0.08476727604327923, 0.005861425303417401)
true shear
(0.0701356992125511, 0.0502658957615494)
measured shear
(0.11038490739617439, 0.001906817913767375)
true shear
(-0.0854376782663166, -0.0281372795812785)
measured shear
(0.03435523423838737, 0.028413169503911408)
true shear
(-0.0416655919980257, 0.0265094874892383)
measured shear
(-0.03576851793005888, 0.03512667211939695)


In [ ]:
# group the dataset by putting the same galaxies together
# iterate over the differrent realizations of the same galaxy
# in each iteration over the same galaxy: make_obs -> perform_metacal -> organize_result -> append to a list
# (a list for different realizations of the same gal) conduct data_index selection of this list -> get_calibrated result by testing ensemble 

# write loop to group dataset
for iy in range(ny):
    for ix in range(nx):
        
        test_id = f"{ix}{iy}"
        this_gal = []
        
        for index, row in TP.iterrows():
            if test_id in row['object_id']:
                this_realization = {
                    'galaxy_image': row['galaxy_image'],
                    'psf_image': row['psf_image'],
                    'shift_radius': row['shift_radius'],
                    'true_shear': row['shear']
                    }
                this_gal.append(this_realization)
        
        this_galaxy = pd.DataFrame(this_gal)
        data_list=[]
        
        for ind, rows in this_galaxy.iterrows():
            galaxy_image = rows['galaxy_image']
            psf_image = rows['psf_image']
            true_shear = rows['true_shear']
            ShiftRadius = rows['shift_radius']
            
            d_x, d_y = ShiftRadius
    
            obs = make_obs(galaxy_image, psf_image, pixel_scale, d_x, d_y)
            resdict, obsdict = perform_metacal(obs, rng_2)
    
            for ShearType, ShearRes in resdict.items():
                dt = organize_result(res=ShearRes, obs=obsdict[ShearType],shear_type=ShearType)
                data_list.append(dt)
                
        data = np.hstack(data_list)
        measured_shear, measured_shear_err = get_calibrated_result(data)
        
        # compute bias
        m1 = measured_shear[0]/true_shear[0]-1
        m2 = measured_shear[1]/true_shear[1]-1
        
        
